In [12]:
import numpy as np
from liberate import fhe
from liberate.fhe import presets
from pprint import pprint

# 1. Generate Parameters

In [2]:
params = presets.params["gold"] # gold 이상이어야함
params["devices"] = [0]
print("|","====="*20)
print("|\tMy params is")
print("|\t", params)
print("|","====="*20)

| ====================================================================================================
|	My params is
|	 {'logN': 16, 'num_special_primes': 4, 'devices': [0]}
| ====================================================================================================


# 2. Generate CKKS engine

In [3]:
%%time

engine = fhe.ckks_engine(**params, verbose=True)

[2023-10-27 12:59:38.015928] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 16
        N				= 65,536
        Number of special primes	= 4
        Number of scales		= 34
        Cache folder			= '/home/kyuyeon/.pyenv/versions/compyler/lib/python3.10/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '1,660' bits out of available maximum '1,661' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516870657, 1099506515969, 1099521458177, 1099503894529, 1099522375681, 1099490000897, 1099523555329, 1099489607681, 1099525128193, 1099486855169, 1099526176769, 1099484889089, 1099529060353, 1099480956929, 1099535220737, 1099469684737, 1099536138241, 1099468767233, 109953

# 3. Generate Keys

In [4]:
%%time

sk = engine.create_secret_key()
pk = engine.create_public_key(sk)
evk = engine.create_evk(sk)
gk = engine.create_galois_key(sk)

CPU times: user 180 ms, sys: 44.8 ms, total: 224 ms
Wall time: 224 ms


# 4. Operations


## 4.1 Mean

In [5]:
var_range = 1
m = engine.example(-var_range, var_range).real
ct = engine.encorypt(m, pk)

ct_mean = engine.mean(ct=ct, gk=gk)
m_mean = engine.decrode(ct_mean, sk)[0].real

print("====="*10)
print(f"|\t\t   HE Mean\t\t\t|")
print("-----"*10)
print(f"|\the\t:\t{m_mean:19.12e}\t|")
print(f"|\torigin\t:\t{m.mean():19.12e}\t|")
print(f"|\terror\t:\t{m_mean - m.mean():19.12e}\t|")
print("====="*10)

|		   HE Mean			|
--------------------------------------------------
|	he	:	-8.589367375619e-04	|
|	origin	:	-8.589302689850e-04	|
|	error	:	-6.468576875854e-09	|


## 4.2 variance (var)

In [6]:
var_range = 1
m = engine.example(-var_range, var_range).real
ct = engine.encorypt(m, pk)

ct_var = engine.var(ct=ct, evk=evk, gk=gk, relin=False)

m_var = engine.decrode(ct_var, sk)[0].real

print("====="*10)
print(f"|\t\t  HE Variacne\t\t\t|")
print("-----"*10)
print(f"|\the\t:\t{m_var:19.12e}\t|")
print(f"|\torigin\t:\t{m.var():19.12e}\t|")
print(f"|\terror\t:\t{m_var - m.var():19.12e}\t|")
print("====="*10)

|		  HE Variacne			|
--------------------------------------------------
|	he	:	 3.350268594660e-01	|
|	origin	:	 3.350268660657e-01	|
|	error	:	-6.599746638081e-09	|


## 4.3 Standard deviation (std)

In [10]:
var_range = 1
m = engine.example(-var_range, var_range).real
ct = engine.encorypt(m, pk)

ct_std = engine.std(ct=ct, evk=evk, gk=gk, relin=False)

m_std = engine.decrode(ct_std, sk)[0].real

print(f"he\t:\t{m_std:19.12e}")
print(f"origin\t:\t{m.std():19.12e}")
print(f"error\t:\t{m_std - m.std():19.12e}")

he	:	 5.765643789746e-01
origin	:	 5.765638973687e-01
error	:	 4.816059395729e-07


In [11]:
print("====="*10)
print(f"|\t\t  HE Standard Devation\t\t|")
print("-----"*10)
print(f"|\the\t:\t{m_std:19.12e}\t|")
print(f"|\torigin\t:\t{m.std():19.12e}\t|")
print(f"|\terror\t:\t{m_std - m.std():19.12e}\t|")
print("====="*10)

|		  HE Standard Devation		|
--------------------------------------------------
|	he	:	 5.765643789746e-01	|
|	origin	:	 5.765638973687e-01	|
|	error	:	 4.816059395729e-07	|
